# How to Save Plots to Files

This comprehensive guide shows you how to export hvPlot visualizations to various file formats for sharing, presentations, or archival purposes.

## Basic HTML Export

The simplest way to save plots is as HTML files, which preserves full interactivity:

In [ ]:
# Suppress Bokeh warnings
import logging
logging.getLogger("bokeh.core.validation.check").setLevel(logging.ERROR)

In [ ]:
import hvplot.pandas # noqa
hvplot.extension('bokeh', 'matplotlib')

penguins = hvplot.sampledata.penguins("pandas").dropna()

plot = penguins.hvplot.scatter(
    x='bill_length_mm',
    y='bill_depth_mm',
    color='species',
    title="Penguin Bill Dimensions"
)

plot

In [ ]:
hvplot.save(plot, 'penguins.html')

## HTML with Inline Resources

By default, HTML files depend on loading JavaScript from online CDN repositories. For offline or airgapped environments, use inline resources:

In [ ]:
from bokeh.resources import INLINE

hvplot.save(plot, 'penguins_offline.html', resources=INLINE)

## PNG Export

To save static PNG images, you need additional dependencies. PNG export requires a browser driver.

### Installing PNG Export Dependencies

Choose one of these options:

**Option 1: Chrome/Chromium (Recommended)**

Install Chrome or Chromium browser first, then install the Python package.

```bash
pip install selenium chromedriver-autoinstaller
```

**Option 2: Firefox**

Install Firefox browser first, then install geckodriver.
```bash
pip install selenium
```
Download geckodriver from [github/geckodriver](https://github.com/mozilla/geckodriver), then add geckodriver to your PATH

**Option 3: Using conda**
```bash
conda install selenium
```

### Saving PNG Files

Once dependencies are installed:

In [ ]:
try:
    hvplot.save(plot, 'penguins.png')
    print("PNG saved successfully as 'penguins.png'")
except Exception as e:
    print(f"PNG export failed: {e}")
    print("Make sure you have selenium and a browser driver installed.")

:::{note}
This will only work if you have selenium and a browser driver installed
:::

### PNG Export Considerations

- **File size**: PNG files are typically larger than HTML for complex plots
- **Interactivity**: PNG files are static - no zoom, pan, or hover tooltips
- **Resolution**: For high-DPI displays or print, consider increasing plot dimensions
- **Performance**: PNG export can be slow for complex plots

In [ ]:
high_res_plot = penguins.hvplot.scatter(
    x='bill_length_mm',
    y='bill_depth_mm',
    width=1200,
    height=800,
    color='species',
    title="High-Res Penguins Plot"
)

high_res_plot

## Backend-Specific Saving

### Bokeh Backend (Default)

Bokeh is the default backend and supports both HTML and PNG export:

In [ ]:
hvplot.output(backend='bokeh')

hvplot.save(plot, 'bokeh_plot.html')

# PNG export (requires browser driver)
try:
    hvplot.save(plot, 'bokeh_plot.png')
    print("Bokeh plot saved as PNG")
except Exception as e:
    print(f"PNG export failed: {e}")

### Matplotlib Backend

Matplotlib backend is excellent for publication-quality static images:

In [ ]:
hvplot.output(backend='matplotlib')

matplotlib_plot = penguins.hvplot.scatter(
    x='bill_length_mm',
    y='bill_depth_mm',
    color='species',
    title="Penguin Bill Dimensions (Matplotlib)",
)

matplotlib_plot

In [ ]:
# Save with matplotlib (PNG export - no browser driver required!)
hvplot.save(matplotlib_plot, 'matplotlib_plot.png')
print("Matplotlib plot saved as PNG")

:::{note} 
Matplotlib doesn't support interactive HTML export. Switch back to Bokeh for HTML exports if needed:

`hvplot.output(backend='bokeh')`
:::

### Matplotlib Advantages for PNG Export

- **No browser driver required**: Matplotlib generates PNG files natively
- **Better text rendering**: Superior font handling for publications
- **Smaller file sizes**: More efficient PNG compression
- **Scientific publishing**: Better suited for academic papers

In [ ]:
import matplotlib
matplotlib.rcParams['figure.dpi'] = 150  # High DPI for better quality

publication_plot = penguins.hvplot.scatter(
    x='bill_length_mm',
    y='bill_depth_mm',
    color='species',
    width=1200,
    height=800,
    title="Penguin Bill Dimensions",
    xlabel="Bill Length (mm)",
    ylabel="Bill Depth (mm)"
)

publication_plot

In [ ]:
hvplot.save(publication_plot, 'publication_figure.png')

## Batch Saving Multiple Plots

You can efficiently save multiple plots in a loop:

In [ ]:
hvplot.output(backend='bokeh')

size_opts = {"frame_width": 250}

plots = {
    'scatter': penguins.hvplot.scatter(x='bill_length_mm', y='bill_depth_mm', color='species', **size_opts),
    'histogram': penguins.hvplot.hist('body_mass_g', by='species', alpha=0.7, **size_opts),
    'box': penguins.hvplot.box(y='flipper_length_mm', by='species', **size_opts),
}

# Display the plots
plots['scatter'] + plots['histogram'] + plots['box']

In [ ]:
for name, plot in plots.items():
    hvplot.save(plot, f'{name}_plot.html')
    print(f"Saved {name}_plot.html")

    # Try PNG export
    try:
        hvplot.save(plot, f'{name}_plot.png')
        print(f"Saved {name}_plot.png")
    except Exception as e:
        print(f"PNG export failed for {name}: {e}")

## Troubleshooting Common Issues

### Testing PNG Export Setup

Here's how to test if your PNG export setup is working:

In [ ]:
# Test if selenium can find your browser
try:
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options

    # Test Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in background
    driver = webdriver.Chrome(options=chrome_options)
    driver.quit()
    print("✓ Chrome driver is working")

except Exception as e:
    print(f"✗ Chrome driver issue: {e}")

    # Try Firefox as fallback
    try:
        from selenium.webdriver.firefox.options import Options as FirefoxOptions
        firefox_options = FirefoxOptions()
        firefox_options.add_argument("--headless")
        driver = webdriver.Firefox(options=firefox_options)
        driver.quit()
        print("✓ Firefox driver is working")
    except Exception as e2:
        print(f"✗ Firefox driver issue: {e2}")
        print("Consider installing browser drivers or using matplotlib backend for PNG export")

### Memory Issues with Large Datasets

For very large datasets, consider using datashader for efficient rendering:

In [ ]:
large_data = hvplot.sampledata.synthetic_clusters("pandas")
print(f"Dataset size: {len(large_data):,} points")

In [ ]:
datashaded_plot = large_data.hvplot.scatter(
    x='x',
    y='y',
    by='cat',
    datashade=True,
    frame_width=300,
    aspect='square',
    title="Large Dataset with Datashader"
)

datashaded_plot

In [ ]:
hvplot.save(datashaded_plot, 'large_dataset.html')
print("Large dataset plot saved successfully")

## Cleanup

Let's clean up the files we created during this demonstration:

In [ ]:
from pathlib import Path

# List of files we created
files_to_clean = [
    'penguins.html', 'penguins_offline.html', 'penguins.png',
    'bokeh_plot.html', 'bokeh_plot.png',
    'matplotlib_plot.png', 'publication_figure.png',
    'scatter_plot.html', 'histogram_plot.html', 'box_plot.html',
    'scatter_plot.png', 'histogram_plot.png', 'box_plot.png',
    'large_dataset.html'
]

for filename in files_to_clean:
    file_path = Path(filename)
    if file_path.exists():
        file_path.unlink()
        print(f"Removed {filename}")
    else:
        print(f"{filename} not found")

## Best Practices Summary

1. **Choose the right format**:
   - **HTML**: For interactive sharing, web embedding
   - **PNG**: For presentations, documents, static sharing
   - **Use matplotlib backend** for publication-quality static images

2. **Test your export pipeline**:
   - Always test both HTML and PNG exports
   - Verify that required dependencies are installed

3. **Consider your audience**:
   - **Technical users**: HTML for full interactivity
   - **Presentations**: PNG for consistent display across platforms
   - **Publications**: Matplotlib PNG for highest quality

4. **Handle large datasets**:
   - Use `rasterize=True` or `datashade=True` for datasets with many points
   - Consider data sampling for very large datasets

5. **Offline environments**:
   - Use `resources=INLINE` for HTML files
   - Matplotlib backend doesn't require internet connectivity

:::{seealso}
- Learn [how to display plots in different environments](display_plots.ipynb)
- Explore [using Panel for advanced layouts and interactivity](use_panel_for_display.ipynb)
:::